# Calculate the expected OOM of OD for Fe in the ISM

In [70]:
import numpy as np
import plotly.express as px
import plotly.offline as po
import xraydb
import plotly.io as pio
pio.renderers.default = "notebook"

Per Liszt, 2013, N(H I)/E(B-V) arXiv:1310.6616v1

Canonically: $N(H) = 5.8\cdot 10^{21} cm^{-2} E(B-V)$

But they argue: $N(H) = 8.3\cdot 10^{21} cm^{-2} E(B-V)$

In this case E(B-V) is in magnitudes.

In [71]:
EBV = 1 # E(B-V) is of the order of magnitude of 1.
NH_EBV = 8.3e21 # atoms of H/cm2

NA = 6.0221409e23 # atoms/mol Avogadro's number

El_H = 8.91e5/2.82e10 # Fe/H ratio from Planetary scientist's compantion table 2.2 (cosmo abundances)

# Given an E(B-V) of one magnitude, then:
NH = NH_EBV*1
print(f'Total number of H atoms in a column with E(B-V)=1: {NH} atoms/cm^2')

# List of elements we want to compute:
HAbundance = 2.82e10  # From Planetary scientist's compantion table 2.2 (cosmo abundances)
Elements = {'H':  {'Abundance': 2.82e10,  'Edge':'K'}, # For each element we have the abundance from table 2.2 and the X-ray edge we want to compute OD for.
            'C':  {'Abundance': 1.00e7,  'Edge':'K'}, 
            'N':  {'Abundance': 2.63e6,  'Edge':'K'},
            'O':  {'Abundance': 2.09e7,  'Edge':'K'},
            'Ne': {'Abundance': 3.31e6,  'Edge':'K'},
            'Mg': {'Abundance': 1.023e6, 'Edge':'K'},
            'Al': {'Abundance': 8.51e4,  'Edge':'K'},
            'Si': {'Abundance': 1.00e6,  'Edge':'K'},
            'S':  {'Abundance': 4.47e5,  'Edge':'K'},
            'Ar': {'Abundance': 1.12e5,  'Edge':'K'},
            'Ca': {'Abundance': 6.646e4, 'Edge':'K'},
            'Cr': {'Abundance': 1.35e4,  'Edge':'K'},
            'Fe-L': {'Abundance': 8.91e5,  'Edge':'L3'},
            'Fe-K': {'Abundance': 8.91e5,  'Edge':'K'},
            'Ni-L': {'Abundance': 5.01e4,  'Edge':'L3'},
            'Ni-K': {'Abundance': 5.01e4,  'Edge':'K'},
           }

Total number of H atoms in a column with E(B-V)=1: 8.3e+21 atoms/cm^2


In [96]:
def CalcOneElement(ElName, ElInfo):
    if '-' in ElName:
        ElName, _ = ElName.split('-')
    print(f'Element {ElName}, mass {xraydb.atomic_mass(ElName)}, Abundance {ElInfo["Abundance"]:g}, Edge {ElInfo["Edge"]}')
    NEl = NH*ElInfo["Abundance"]/Elements['H']['Abundance']
    print(f'\tTotal number of {ElName} atoms in column: {NEl} atoms/cm^2')
    rhot = NEl * xraydb.atomic_mass(ElName) / NA
    print(f'\trho*t = {rhot}')    
    energies = np.linspace(-10, 50, 60) + xraydb.xray_edge(ElName, ElInfo["Edge"]).energy
    mu = xraydb.mu_elam(ElName, energies)   
    # Calculate the pre and post edge transmission: I/I0 = exp(-mu rho tau)
    Ipre = np.exp(-np.min(mu)*rhot)
    Ipost = np.exp(-np.max(mu)*rhot)
    OD = 1 - Ipost/Ipre
    print(f'\tExpected OD of {ElName}-{ElInfo["Edge"]} for an E(B-V) ~ 1 is {OD}')
    TableStr = f"{ElName} & {ElInfo['Abundance']/Elements['H']['Abundance']:0.2g} & {ElInfo['Edge']} & {xraydb.xray_edge(ElName, ElInfo['Edge']).energy} & {OD:0.1} \\\\\n"
    po.iplot(px.line(x=energies,y=mu, title=f'mu in cm2/g for {ElName}-{ElInfo["Edge"]}', labels={'x':'eV', 'y':'mu'}))
    print('\n')
    return TableStr
    
TableStr = ('\\begin{table}[]\n'
            '\\begin{tabular}{lllll}\n'
            'Element & Abundance ($N_{El}/N_H$) & Edge & Energy (eV) & OD \\\\\n'
            '\\hline\n')

for ElName, ElInfo in Elements.items():
    TableStr += CalcOneElement(ElName, ElInfo)


TableStr += ('\\end{tabular}\n'
             '\\end{table}\n')
print(TableStr)

Element H, mass 1.0078, Abundance 2.82e+10, Edge K
	Total number of H atoms in column: 8.299999999999999e+21 atoms/cm^2
	rho*t = 0.013889977233843864
	Expected OD of H-K for an E(B-V) ~ 1 is 0.0




Element C, mass 12.011, Abundance 1e+07, Edge K
	Total number of C atoms in column: 2.9432624113475174e+18 atoms/cm^2
	rho*t = 5.870258668755995e-05
	Expected OD of C-K for an E(B-V) ~ 1 is 0.930697443194843




Element N, mass 14.007, Abundance 2.63e+06, Edge K
	Total number of N atoms in column: 7.740780141843972e+17 atoms/cm^2
	rho*t = 1.800441225923633e-05
	Expected OD of N-K for an E(B-V) ~ 1 is 0.3755316907762196




Element O, mass 15.999, Abundance 2.09e+07, Edge K
	Total number of O atoms in column: 6.151418439716312e+18 atoms/cm^2
	rho*t = 0.0001634245117330637
	Expected OD of O-K for an E(B-V) ~ 1 is 0.9419690242094868




Element Ne, mass 20.1797, Abundance 3.31e+06, Edge K
	Total number of Ne atoms in column: 9.742198581560283e+17 atoms/cm^2
	rho*t = 3.264530803593653e-05
	Expected OD of Ne-K for an E(B-V) ~ 1 is 0.2531547968969782


...



Element Fe, mass 55.845, Abundance 891000, Edge L3
	Total number of Fe atoms in column: 2.622446808510638e+17 atoms/cm^2
	rho*t = 2.4318684078161735e-05
	Expected OD of Fe-L3 for an E(B-V) ~ 1 is 0.26197331824981407




Element Fe, mass 55.845, Abundance 891000, Edge K
	Total number of Fe atoms in column: 2.622446808510638e+17 atoms/cm^2
	rho*t = 2.4318684078161735e-05
	Expected OD of Fe-K for an E(B-V) ~ 1 is 0.008582931339726652




Element Ni, mass 58.6934, Abundance 50100, Edge L3
	Total number of Ni atoms in column: 1.4745744680851064e+16 atoms/cm^2
	rho*t = 1.4371598161229734e-06
	Expected OD of Ni-L3 for an E(B-V) ~ 1 is 0.012908480270206302




Element Ni, mass 58.6934, Abundance 50100, Edge K
	Total number of Ni atoms in column: 1.4745744680851064e+16 atoms/cm^2
	rho*t = 1.4371598161229734e-06
	Expected OD of Ni-K for an E(B-V) ~ 1 is 0.00040955257997343786




\begin{table}[]
\begin{tabular}{lllll}
Element & Abundance ($N_{El}/N_H$) & Edge & Energy (eV) & OD \\
\hline
H & 1 & K & 13.6 & 0e+00 \\
C & 0.00035 & K & 284.2 & 0.9 \\
N & 9.3e-05 & K & 409.9 & 0.4 \\
O & 0.00074 & K & 543.1 & 0.9 \\
Ne & 0.00012 & K & 870.2 & 0.3 \\
Mg & 3.6e-05 & K & 1303.0 & 0.06 \\
Al & 3e-06 & K & 1559.0 & 0.004 \\
Si & 3.5e-05 & K & 1839.0 & 0.04 \\
S & 1.6e-05 & K & 2472.0 & 0.01 \\
Ar & 4e-06 & K & 3205.9 & 0.002 \\
Ca & 2.4e-06 & K & 4038.5 & 0.001 \\
Cr & 4.8e-07 & K & 5989.0 & 0.0002 \\
Fe & 3.2e-05 & L3 & 706.8 & 0.3 \\
Fe & 3.2e-05 & K & 7112.0 & 0.009 \\
Ni & 1.8e-06 & L3 & 852.7 & 0.01 \\
Ni & 1.8e-06 & K & 8333.0 & 0.0004 \\
\end{tabular}
\end{table}



In [97]:
!jupyter nbconvert --to html "Calculate OOM of ISM OD.ipynb"

[NbConvertApp] Converting notebook Calculate OOM of ISM OD.ipynb to html
[NbConvertApp] Writing 488581 bytes to Calculate OOM of ISM OD.html
